In [5]:
import requests as rq
from bs4 import BeautifulSoup as bs
import pandas as pd

def convert_to_number(s):
    # Remove commas from the string
    s = s.replace(",", "")
    # Convert to float if there's a decimal point, otherwise to convert_to_number
    return float(s) if '.' in s else int(s)

print('-------------- Mahfuz Special Search ----------------\n')
ordersdf = pd.read_excel('orders.xlsx')

df = pd.DataFrame()
columns = ["Buyer", "Order", 'Style',	'UoF',	'F. Color',	'G. Color', 'Y. Type', 'F. Type', 'GSM', 'Dia',	'G/F Order With S.Note Qty', 'G/F S.Note Qty',	'Net Grey Receive Qty',	'G/F Rcv Balance Qty',	'F/F Order with S.Note Qty',	'F/F S.Note Qty',	'F/F Delv Qty',	'F/F Delv. Balance Qty',	'Replacement Delivery',	'F/F Excess Delv.Qty',	'Transfer To',	'Transfer From',	'Return Receive',	'Return Delivery',	'Dyeing Unit',	'Order Sheet Receive Date', 'Cut Plan Start Date',	'Cut Plan End Date', 'Unit Store']
df = df.reindex(columns=df.columns.tolist() + columns)

row_idx = 0
no_of_orders = len(ordersdf['FRS No.'])

for index, r in ordersdf.iterrows():
    order = r['FRS No.']
    print("Process completed", int((index + 1) * 100 / no_of_orders), "%  <---->", 'FRS:', order)
    # print("\nCalculating for order: ", order)
    url = 'http://192.168.13.253/mymun/Work%20Order/combineSearchResult.php?Welcome=7&GetOrderNO=' + str(order)
    response = rq.get(url)

    html_content = bs(response.content, 'html.parser')
    rows = []
    for row in html_content.find_all('tr'):
        row_data = [cell.get_text(strip=True) for cell in row.find_all('td')]
        rows.append(row_data)

    row_idx2 = row_idx
    si = 1
    for row in rows:
        if len(row) > 0 and row[0].isnumeric() and row[0] != '0':
            if int(row[0]) < si:
                break
            if convert_to_number(row[8]) == 0:
                continue
            df.loc[row_idx, 'Buyer'] = rows[3][0]
            df.loc[row_idx, 'Order'] = rows[3][1]
            df.loc[row_idx, 'Style'] = rows[3][2]
            df.loc[row_idx, 'UoF'] = row[1]
            df.loc[row_idx, 'F. Color'] = row[2][0:row[2].find('(')]
            df.loc[row_idx, 'G. Color'] = row[3]
            df.loc[row_idx, 'Y. Type'] = row[4]
            df.loc[row_idx, 'F. Type'] = row[5]
            df.loc[row_idx, 'GSM'] = convert_to_number(row[6])
            df.loc[row_idx, 'Dia'] = row[7]
            df.loc[row_idx, 'G/F Order With S.Note Qty'] = convert_to_number(row[8])
            df.loc[row_idx, 'G/F S.Note Qty'] = convert_to_number(row[9])
            df.loc[row_idx, 'Net Grey Receive Qty'] = convert_to_number(row[13])
            df.loc[row_idx, 'G/F Rcv Balance Qty'] = convert_to_number(row[15])
            row_idx += 1
            si += 1
    
    row_idx = row_idx2
    si = 0
    for row in rows:
        if len(row) == 1:
                if row[0].startswith('Define Unit::'):
                    df.loc[row_idx, 'Unit Store'] = row[0].split('::')[1]
                if row[0].startswith('Finish Fabric Delivery'):
                    df.loc[row_idx, 'Unit Store'] = 'Jinnat'
        if len(row) > 0 and row[0] == 'SL NO.':
            si += 1
        if len(row) > 0 and si == 2 and row[0].isnumeric() and row[0] != '0':
            if convert_to_number(row[8]) == 0:
                continue
            df.loc[row_idx, 'F/F Order with S.Note Qty'] = convert_to_number(row[8])
            df.loc[row_idx, 'F/F S.Note Qty'] = convert_to_number(row[9])
            df.loc[row_idx, 'F/F Delv Qty'] = convert_to_number(row[12])
            df.loc[row_idx, 'F/F Delv. Balance Qty'] = convert_to_number(row[15])
            df.loc[row_idx, 'Replacement Delivery'] = convert_to_number(row[16])
            df.loc[row_idx, 'F/F Excess Delv.Qty'] = convert_to_number(row[18])
            df.loc[row_idx, 'Transfer To'] = row[19]
            df.loc[row_idx, 'Transfer From'] = row[20]
            df.loc[row_idx, 'Return Receive'] = convert_to_number(row[22])
            df.loc[row_idx, 'Return Delivery'] = convert_to_number(row[23])
            df.loc[row_idx, 'Dyeing Unit'] = row[26]
            df.loc[row_idx, 'Order Sheet Receive Date'] = rows[3][4]
            df.loc[row_idx, 'Cut Plan Start Date'] = rows[3][5]
            df.loc[row_idx, 'Cut Plan End Date'] = rows[3][6]
            row_idx += 1

-------------- Mahfuz Special Search ----------------

Process completed 33 %  <----> FRS: 2025/3985


C:\Users\Mahafuz\AppData\Local\Temp\ipykernel_10868\2102165318.py:42: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'GEORGE' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[row_idx, 'Buyer'] = rows[3][0]
C:\Users\Mahafuz\AppData\Local\Temp\ipykernel_10868\2102165318.py:43: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2025/3985' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[row_idx, 'Order'] = rows[3][1]
C:\Users\Mahafuz\AppData\Local\Temp\ipykernel_10868\2102165318.py:44: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'BGSPQ1195' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[row_idx, 'Style']

Process completed 66 %  <----> FRS: 2025/13605
Process completed 100 %  <----> FRS: 2025/6477


In [9]:
print("Calculating Combo Status ...")
combo = pd.DataFrame()
combo_columns = ['Buyer', 'Order', 'Style', 'UoF', 'F. Color', 'G. Color', 'F. Type', 'F/F Order with S.Note Qty',	'F/F S.Note Qty',	'F/F Delv Qty',	'F/F Delv. Balance Qty',	'Replacement Delivery',	'F/F Excess Delv.Qty',	'Transfer To',	'Transfer From',	'Return Receive',	'Return Delivery',	'Dyeing Unit',	'Order Sheet Receive Date', 'Cut Plan Start Date',	'Cut Plan End Date', 'Unit Store']
combo = combo.reindex(columns=combo.columns.tolist() + combo_columns)
dev_tolerance = 0.1
combo = {}
for index, row in df.iterrows():
    order = row['Order']
    if row['G. Color'] == 'Multi' or row['G. Color'] == '':
        color = row['F. Color']
        color = color.split(" AOP")[0] if " AOP" in color else color
    else:
        color = row['G. Color']
    booking_qty = float(row['F/F Order with S.Note Qty'])
    receive_qty = float(row['F/F Delv Qty'])
    rec_pctg = min(1.0, round((receive_qty / booking_qty) if booking_qty != 0 else 0, 2))
    F_type = row['F. Type']
    if order not in combo:
        combo[order] = {}
    if color not in combo[order]:
        combo[order][color] = []
    
    combo[order][color].append([booking_qty, receive_qty, rec_pctg, F_type])
print(combo)

Calculating Combo Status ...
{'2025/3985': {'11-0607 Tcx Sugar Swizzle': [[1349.0, 1478.5, 1.0, 'SLUB S/J'], [1287.0, 1114.1, 0.87, 'SLUB S/J']], '17-3918 Tcx-Country Blue': [[1215.0, 1187.0, 0.98, 'SLUB S/J']], '19-3921 TCX-BLACK IRIS': [[1215.0, 1189.0, 0.98, 'SLUB S/J']], '19-1535 Tcx-Syrah': [[1215.0, 1252.3, 1.0, 'SLUB S/J']], '11-0607 Tcx Sugar Swizzle ': [[43.0, 60.0, 1.0, '1X1 FULL FEEDER RIB'], [43.0, 41.0, 0.95, '1X1 FULL FEEDER RIB']], '17-3918 Tcx-Country Blue ': [[43.0, 0.0, 0.0, '1X1 FULL FEEDER RIB']], '19-3921 Tcx-Black Irsh ': [[43.0, 36.0, 0.84, '1X1 FULL FEEDER RIB']], '19-1535 Tcx-Syrah ': [[43.0, 35.0, 0.81, '1X1 FULL FEEDER RIB']]}, '2025/13605': {'Classic Black ': [[3886.0, 0.0, 0.0, 'L S/J']], '11-0601 TCX BRIGHT WHITE ': [[4549.0, 0.0, 0.0, 'L S/J']], 'SNOW WHITE 11-0602 TCX ': [[17464.0, 0.0, 0.0, 'INTERLOCK'], [873.0, 0.0, 0.0, '1x1 RIB']], '232482 Classic Black ': [[9196.0, 0.0, 0.0, 'INTERLOCK'], [461.0, 0.0, 0.0, '1x1 RIB']]}, '2025/6477': {'14-3949 Tcx-Xe

In [10]:
import json

with open('combo_output.json', 'w', encoding='utf-8') as f:
    json.dump(combo, f, indent=4, ensure_ascii=False)